In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import dateutil.parser
from datetime import timedelta, datetime
from pytz import timezone
import pytz
from random import randint
from time import sleep
from torrequest import TorRequest

In [3]:
'''Package what we did above into a function'''
def scrape_news_text(news_url):
     
    print(news_url)

    news_html = requests.get(news_url).content
 
    '''convert html to BeautifulSoup object'''
    news_soup = BeautifulSoup(news_html , 'lxml')
    
    timestamp = news_soup.find("meta", {"property":"article:published_time"})['content']
    
    if(timestamp == None):
        timestamp = news_soup.find_all("small")[0].find("span")['content']
    
    print(timestamp)
    
    if(timestamp):
        try:
            timestamp_to_datetime = dateutil.parser.parse(str(timestamp))
            print(timestamp_to_datetime)
            convert_to_east_western = timestamp_to_datetime + timedelta(hours=7)
            print(convert_to_east_western)
            print(convert_to_east_western.timestamp())
            timestamp_decode = datetime.utcfromtimestamp(convert_to_east_western.timestamp()).strftime('%Y-%m-%d %H:%M:%S')
            titles = news_soup.find('h1').text
            paragraphs = [par.text for par in news_soup.find_all('p')]
            paragraphs = paragraphs[1:len(paragraphs)-6]
            print(timestamp_decode)

        except:
            print(news_soup)
            titles = None
            timestamp = None
            convert_to_east_western = None
            timestamp_decode = None
            news_text = None
            pass

        print(titles)
        print("")

        news_text = '\n'.join(paragraphs)

        return {'title':titles, 
                'datePublished': timestamp, 'url':news_url,
                'unix_timestamp': convert_to_east_western.timestamp(),
                'timestamp_decode' : timestamp_decode,
                'text': news_text}
    else:
        return {'title':None, 
                'datePublished': None, 'url':None,
                'unix_timestamp': None,
                'timestamp_decode' : None,
                'text': None}

In [34]:
'''Generalized function to get all news-related articles from a Nasdaq webpage'''
def get_news_urls(links_site):
    
    print(links_site)
    
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
        
    
    '''scrape the html of the site'''
    resp = tr.get(links_site, headers=headers, timeout=30)
 
    if not resp.ok:
        return None
 
    html = resp.content
 
    '''convert html to BeautifulSoup object'''
    soup = BeautifulSoup(html , 'lxml')
 
    '''get list of all links on webpage'''
    #links = soup.find_all('a')
    links = soup.find_all(target="_self")
 
    urls = [link.get('href') for link in links]
    urls = [url for url in urls if url is not None]
 
    '''Filter the list of urls to just the news articles'''
    news_urls = [url for url in urls if '/article/' in url]
 
    return news_urls

In [5]:
def scrape_all_articles(ticker, upper_page_limit, index):
 
    landing_site = 'http://www.nasdaq.com/symbol/' + ticker + '/news-headlines'
    
    print(landing_site)
    
    if(index == 2):
        all_news_urls = get_news_urls(landing_site)
        current_urls_list = all_news_urls.copy()
    else:
        all_news_urls = []
        
    #index = 2
    
    '''Loop through each sequential page, scraping the links from each'''
    #while (current_urls_list is not None) and (current_urls_list != []) and \
    while (index <= upper_page_limit):
         
        #print(index)
        
        '''Construct URL for page in loop based off index'''
        current_site = landing_site + '?page=' + str(index)
        current_urls_list = get_news_urls(current_site)
        
        print(current_urls_list)
         
        '''Append current webpage's list of urls to all_news_urls'''
        all_news_urls = all_news_urls + current_urls_list

        index = index + 1
        
        sleep(randint(30,60))
 
    all_news_urls = list(set(all_news_urls))
     
    '''Now, we have a list of urls, we need to actually scrape the text'''
    #all_articles = [scrape_news_text(news_url) for news_url in all_news_urls]
    
    all_articles = []
    for i in all_news_urls:
        all_articles.append(scrape_news_text(i))
        
    return all_articles

In [70]:
# function test
max_page = 140
page_end = 110
page_start = 106
while(page_end <= max_page):
    print("start page: " + str(page_start) + ", end page: " + str(page_end))
    print(page_end)
    nflx_articles = scrape_all_articles('nflx' , page_end, page_start)
    df = pd.DataFrame(nflx_articles)
    df.to_csv('FB_nasdaq_news_' + str(page_start) + '_' + str(page_end) + '.csv', encoding='utf-8')
    page_start = page_start + 5
    page_end = page_end + 5
    sleep(randint(30,60))

start page: 106, end page: 110
110
http://www.nasdaq.com/symbol/nflx/news-headlines
http://www.nasdaq.com/symbol/nflx/news-headlines?page=106
['https://www.nasdaq.com/article/upcoming-tech-earnings-to-watch-msft-amzn-googl-cm1039939', 'https://www.nasdaq.com/article/is-amazon-stock-a-bubble-right-now-cm1039999', 'https://www.nasdaq.com/article/7-breakout-stocks-to-buy-that-are-immune-to-volatility-cm1039957', 'https://www.nasdaq.com/article/the-zacks-analyst-blog-highlights-amazon-walmart-google-facebook-and-netflix-cm1039853', 'https://www.nasdaq.com/article/now-that-netflix-stock-has-chilled-its-looking-binge-worthy-again-cm1039958', 'https://www.nasdaq.com/article/stock-market-news-for-oct-19-2018-cm1039678', 'https://www.nasdaq.com/article/where-should-investors-go-to-find-out-whats-most-important-to-a-company-cm1039722', 'https://www.nasdaq.com/article/the-1-netflix-number-that-matters-7-million-cm1039727', 'https://www.nasdaq.com/article/netflix-keeps-surging-ibm-stops-surging-cm

2018-10-17T01:06:31-05:00
2018-10-17 01:06:31-05:00
2018-10-17 08:06:31-05:00
1539781591.0
2018-10-17 13:06:31
Stocks Jump 2%+ on Solid Earnings

https://www.nasdaq.com/article/netflix-stock-skyrockets-on-q3-earnings-beat-cm1038247
2018-10-16T04:46:48-05:00
2018-10-16 04:46:48-05:00
2018-10-16 11:46:48-05:00
1539708408.0
2018-10-16 16:46:48
Netflix Stock Skyrockets on Q3 Earnings Beat

https://www.nasdaq.com/article/sp-500-movers-ntrs-nflx-cm1038482
2018-10-17T10:17:04-05:00
2018-10-17 10:17:04-05:00
2018-10-17 17:17:04-05:00
1539814624.0
2018-10-17 22:17:04
S&P 500 Movers: NTRS, NFLX

https://www.nasdaq.com/article/consumer-sector-update-for-10172018-czrnflxwgotaco-cm1038703
2018-10-17T03:58:04-05:00
2018-10-17 03:58:04-05:00
2018-10-17 10:58:04-05:00
1539791884.0
2018-10-17 15:58:04
Consumer Sector Update for 10/17/2018: CZR,NFLX,WGO,TACO

https://www.nasdaq.com/article/netflix-nflx-beats-on-q3-earnings-subscriber-addition-up-cm1038672
2018-10-17T02:32:00-05:00
2018-10-17 02:32:00-05

2018-10-17T05:33:00-05:00
2018-10-17 05:33:00-05:00
2018-10-17 12:33:00-05:00
1539797580.0
2018-10-17 17:33:00
Bundling Continues to Boost Netflix's Subscriber Growth

https://www.nasdaq.com/article/the-zacks-analyst-blog-highlights-google-facebook-amazon-and-netflix-cm1039006
2018-10-18T09:33:00-05:00
2018-10-18 09:33:00-05:00
2018-10-18 16:33:00-05:00
1539898380.0
2018-10-18 21:33:00
The Zacks Analyst Blog Highlights: Google, Facebook, Amazon and Netflix

https://www.nasdaq.com/article/netflix-nflx-and-american-airlines-aal-earnings-confirm-strong-bull-cases-cm1038458
2018-10-17T10:26:44-05:00
2018-10-17 10:26:44-05:00
2018-10-17 17:26:44-05:00
1539815204.0
2018-10-17 22:26:44
Netflix (NFLX) And American Airlines (AAL) Earnings Confirm Strong Bull Cases

https://www.nasdaq.com/article/buy-canopy-growth-as-pot-goes-legal-in-canada-i-wouldnt-cm1038685
2018-10-17T11:32:58-05:00
2018-10-17 11:32:58-05:00
2018-10-17 18:32:58-05:00
1539819178.0
2018-10-17 23:32:58
Buy Canopy Growth as Pot 

2018-10-15T08:21:00-05:00
2018-10-15 08:21:00-05:00
2018-10-15 15:21:00-05:00
1539634860.0
2018-10-15 20:21:00
Twilio, Delphi Technologies, Netflix, IBM and PayPal Holdings highlighted as Zacks Bull and Bear of the Day

https://www.nasdaq.com/article/go-long-jpmorgan-chase-stock-on-earnings-cm1036843
2018-10-12T01:14:49-05:00
2018-10-12 01:14:49-05:00
2018-10-12 08:14:49-05:00
1539350089.0
2018-10-12 13:14:49
Go Long JPMorgan Chase Stock on Earnings

https://www.nasdaq.com/article/is-the-sharing-economy-stunting-netflixs-growth-cm1036945
2018-10-14T09:30:00-05:00
2018-10-14 09:30:00-05:00
2018-10-14 16:30:00-05:00
1539552600.0
2018-10-14 21:30:00
Is the Sharing Economy Stunting Netflix's Growth?

https://www.nasdaq.com/article/we-now-know-how-apple-plans-to-challenge-netflix-and-amazon-cm1037558
2018-10-15T09:48:00-05:00
2018-10-15 09:48:00-05:00
2018-10-15 16:48:00-05:00
1539640080.0
2018-10-15 21:48:00
We Now Know How Apple Plans to Challenge Netflix and Amazon

https://www.nasdaq.co

2018-10-12T08:56:47-05:00
2018-10-12 08:56:47-05:00
2018-10-12 15:56:47-05:00
1539377807.0
2018-10-12 20:56:47
Here's Some Backstory to Netflix's Deal to Buy ABQ Studios, Where "Better Call Saul" Is Produced

https://www.nasdaq.com/article/3-things-that-can-go-wrong-for-netflix-next-week-cm1036265
2018-10-12T08:24:00-05:00
2018-10-12 08:24:00-05:00
2018-10-12 15:24:00-05:00
1539375840.0
2018-10-12 20:24:00
3 Things That Can Go Wrong for Netflix Next Week

https://www.nasdaq.com/article/netflix-nflx-buys-movie-rights-of-dwayne-johnson-starrer-cm1037325
2018-10-15T10:04:00-05:00
2018-10-15 10:04:00-05:00
2018-10-15 17:04:00-05:00
1539641040.0
2018-10-15 22:04:00
Netflix (NFLX) Buys Movie Rights of Dwayne Johnson Starrer

https://www.nasdaq.com/article/why-netflix-stock-jumped-friday-cm1036653
2018-10-12T01:04:00-05:00
2018-10-12 01:04:00-05:00
2018-10-12 08:04:00-05:00
1539349440.0
2018-10-12 13:04:00
Why Netflix Stock Jumped Friday

https://www.nasdaq.com/article/5-of-the-best-stocks-to

...

2018-09-27T09:20:42-05:00
2018-09-27 09:20:42-05:00
2018-09-27 16:20:42-05:00
1538083242.0
2018-09-27 21:20:42
Thursday’s Vital Data: Square, Netflix and Fortive

https://www.nasdaq.com/article/media-merger-mania-fox-sells-sky-stake-cm1028786
2018-09-26T03:54:00-05:00
2018-09-26 03:54:00-05:00
2018-09-26 10:54:00-05:00
1537977240.0
2018-09-26 15:54:00
Media Merger Mania: Fox Sells Sky Stake

https://www.nasdaq.com/article/whats-in-store-for-faang-stocks-during-q3-earnings-season-cm1028005
2018-09-25T05:30:00-05:00
2018-09-25 05:30:00-05:00
2018-09-25 12:30:00-05:00
1537896600.0
2018-09-25 17:30:00
What's In Store for FAANG Stocks During Q3 Earnings Season?

https://www.nasdaq.com/article/discovery-disca-hits-52-week-high-on-portfolio-strength-cm1031648
2018-10-03T11:05:00-05:00
2018-10-03 11:05:00-05:00
2018-10-03 18:05:00-05:00
1538607900.0
2018-10-03 23:05:00
Discovery (DISCA) Hits 52-Week High on Portfolio Strength

https://www.nasdaq.com/article/an-early-netflix-nflx-q3-earnings-pr

http://www.nasdaq.com/symbol/nflx/news-headlines?page=128
['https://www.nasdaq.com/article/wall-streets-reefer-madness-micron-earnings-more-stock-news-cm1026471', 'https://www.nasdaq.com/article/3-high-dividend-stocks-to-buy-for-safety-in-your-portfolio-cm1027208', 'https://www.nasdaq.com/article/netflix-stock-is-ready-to-rally-higher-cm1027216', 'https://www.nasdaq.com/article/remember-that-theres-more-to-disney-stock-than-just-streaming-cm1027217', 'https://www.nasdaq.com/article/top-analyst-reports-for-netflix-eog-resources-simon-property-group-cm1026414', 'https://www.nasdaq.com/article/consumers-comfort-rises-as-economy-gains-steam-5-winners-cm1026398', 'https://www.nasdaq.com/article/comcast-fox-set-to-decide-skys-fate-in-three-round-auction-cm1026281', 'https://www.nasdaq.com/article/usa-technologies-usat-q4-earnings-whats-in-the-cards-cm1026239', 'https://www.nasdaq.com/article/media-war-for-streaming-heats-up-fox-comcast-bid-for-sky-cm1026135', 'https://www.nasdaq.com/article/

2018-09-24T01:34:25-05:00
2018-09-24 01:34:25-05:00
2018-09-24 08:34:25-05:00
1537796065.0
2018-09-24 13:34:25
Can IBM Stock Be Boosted by the Cloud, Blockchain, And AI?

https://www.nasdaq.com/article/the-10-best-stocks-to-buy-right-now-cm1027324
2018-09-24T12:39:26-05:00
2018-09-24 12:39:26-05:00
2018-09-24 19:39:26-05:00
1537835966.0
2018-09-25 00:39:26
The 10 Best Stocks to Buy Right Now

https://www.nasdaq.com/article/market-close-report-nasdaq-composite-index-closes-at-795611-up-6032-points-cm1024652
2018-09-18T04:45:10-05:00
2018-09-18 04:45:10-05:00
2018-09-18 11:45:10-05:00
1537289110.0
2018-09-18 16:45:10
Market Close Report: NASDAQ Composite index closes at 7,956.11 up 60.32 points

https://www.nasdaq.com/article/why-did-disney-dis-netflix-nflx-stock-jump-monday-cm1027237
2018-09-24T02:22:00-05:00
2018-09-24 02:22:00-05:00
2018-09-24 09:22:00-05:00
1537798920.0
2018-09-24 14:22:00
Why Did Disney (DIS) & Netflix (NFLX) Stock Jump Monday?

https://www.nasdaq.com/article/why-di

http://www.nasdaq.com/symbol/nflx/news-headlines?page=132
['https://www.nasdaq.com/article/apples-health-fitness-push-plus-nikes-latest-global-move-cm1023949', 'https://www.nasdaq.com/article/why-the-emmys-matter-so-much-for-netflix-nflx-cm1023924', 'https://www.nasdaq.com/article/a-closer-look-at-comcasts-valuation-cm1023811', 'https://www.nasdaq.com/article/why-netflix-nflx-could-beat-earnings-estimates-again-cm1023774', 'https://www.nasdaq.com/article/everyone-but-disney-wants-to-scare-you-this-weekend-cm1023356', 'https://www.nasdaq.com/article/imax-expands-presence-in-china-with-multiple-partnerships-cm1023287', 'https://www.nasdaq.com/article/buy-netflix-stock-on-recent-content-strength-cm1023254', 'https://www.nasdaq.com/article/3-tech-stocks-with-more-bite-than-fang-cm1023256', 'https://www.nasdaq.com/article/3-big-stock-charts-for-friday-comcast-lockheed-martin-and-ppl-cm1023080', 'https://www.nasdaq.com/article/netflix-nflx-stock-sinks-as-market-gains-what-you-should-know-cm1

http://www.nasdaq.com/symbol/nflx/news-headlines?page=137
['https://www.nasdaq.com/article/apple-aapl-stock-is-a-strong-buy-way-beyond-the-iphone-xs-max-cm1022715', 'https://www.nasdaq.com/article/new-iphones-amd-gains-news-from-nflx-etsy-tlrd-free-lunch-cm1022649', 'https://www.nasdaq.com/article/bear-of-the-day-tencent-tcehy-cm1022230']
http://www.nasdaq.com/symbol/nflx/news-headlines?page=138
['https://www.nasdaq.com/article/apple-aapl-stock-is-a-strong-buy-way-beyond-the-iphone-xs-max-cm1022715', 'https://www.nasdaq.com/article/new-iphones-amd-gains-news-from-nflx-etsy-tlrd-free-lunch-cm1022649', 'https://www.nasdaq.com/article/bear-of-the-day-tencent-tcehy-cm1022230']
http://www.nasdaq.com/symbol/nflx/news-headlines?page=139
['https://www.nasdaq.com/article/apple-aapl-stock-is-a-strong-buy-way-beyond-the-iphone-xs-max-cm1022715', 'https://www.nasdaq.com/article/new-iphones-amd-gains-news-from-nflx-etsy-tlrd-free-lunch-cm1022649', 'https://www.nasdaq.com/article/bear-of-the-day-ten

In [56]:
df.head()

,datePublished,text,timestamp_decode,title,unix_timestamp,url
0,2019-01-20T11:38:00-05:00,"\nOver the past several years, the prevailing...",2019-01-20 23:38:00,"Streaming Isn't Killing Theaters, Contrary to ...",1.548027e+09,https://www.nasdaq.com/article/streaming-isnt-...
1,2019-01-18T09:03:00-05:00,\n Texas Instruments TXN or TI is scheduled ...,2019-01-18 21:03:00,Can Analog Strength Aid Texas Instruments (TXN...,1.547845e+09,https://www.nasdaq.com/article/can-analog-stre...
2,2019-01-21T08:31:00-05:00,\n STMicroelectronics N.V. STM is slated to ...,2019-01-21 20:31:00,What's in Store for STMicroelectronics (STM) i...,1.548103e+09,https://www.nasdaq.com/article/whats-in-store-...
3,2019-01-21T05:18:00-05:00,\nShares of AT&T T have surged over 13% sinc...,2019-01-21 17:18:00,AT&T (T) Q4 Earnings Preview,1.548091e+09,https://www.nasdaq.com/article/atampt-t-q4-ear...
4,2019-01-18T06:59:00-05:00,\nIf you're interested in broad exposure to t...,2019-01-18 18:59:00,Should John Hancock Multifactor Large Cap ETF ...,1.547838e+09,https://www.nasdaq.com/article/should-john-han...


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
datePublished       50 non-null object
text                50 non-null object
timestamp_decode    50 non-null object
title               50 non-null object
unix_timestamp      50 non-null float64
url                 50 non-null object
dtypes: float64(1), object(5)
memory usage: 2.4+ KB


# Concat all the .csv file in the same folder

In [42]:
import glob
import os
os.getcwd()

'/Users/LarryWong/Desktop/Accelerate Tech/capstone/study_stocks_sentiments'

In [74]:
folder = os.getcwd()
path = os.getcwd() + "/FB"
allFiles = glob.glob(path + "/*.csv")

list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)

frame = pd.concat(list_, axis = 0, ignore_index = True)

In [75]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326 entries, 0 to 1325
Data columns (total 7 columns):
Unnamed: 0          1326 non-null int64
datePublished       1324 non-null object
text                1322 non-null object
timestamp_decode    1324 non-null object
title               1324 non-null object
unix_timestamp      1324 non-null float64
url                 1324 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 72.6+ KB


In [76]:
frame = frame.drop(labels=["Unnamed: 0", "timestamp_decode", "unix_timestamp"], axis=1)

In [77]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326 entries, 0 to 1325
Data columns (total 4 columns):
datePublished    1324 non-null object
text             1322 non-null object
title            1324 non-null object
url              1324 non-null object
dtypes: object(4)
memory usage: 41.5+ KB


In [78]:
cleaned_frame = frame.dropna(subset=["text"])

In [79]:
cleaned_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1322 entries, 0 to 1325
Data columns (total 4 columns):
datePublished    1322 non-null object
text             1322 non-null object
title            1322 non-null object
url              1322 non-null object
dtypes: object(4)
memory usage: 51.6+ KB


In [80]:
cleaned_frame.sort_values(by="datePublished", ascending=True)

,datePublished,text,title,url
371,2018-09-13T01:18:00-05:00,"\n \nOn today's episode of Free Lunch, Asso...","New iPhones, AMD Gains, & News from NFLX, ETSY...",https://www.nasdaq.com/article/new-iphones-amd...
374,2018-09-13T01:18:00-05:00,"\n \nOn today's episode of Free Lunch, Asso...","New iPhones, AMD Gains, & News from NFLX, ETSY...",https://www.nasdaq.com/article/new-iphones-amd...
373,2018-09-13T03:22:00-05:00,\nApple AAPL saw its stock price jump nearly...,Apple (AAPL) Stock Is A Strong Buy Way Beyond ...,https://www.nasdaq.com/article/apple-aapl-stoc...
351,2018-09-13T03:22:00-05:00,\nApple AAPL saw its stock price jump nearly...,Apple (AAPL) Stock Is A Strong Buy Way Beyond ...,https://www.nasdaq.com/article/apple-aapl-stoc...
365,2018-09-13T05:45:16-05:00,\nNetflix (NFLX) closed the most recent tradi...,Netflix (NFLX) Stock Sinks As Market Gains: Wh...,https://www.nasdaq.com/article/netflix-nflx-st...
375,2018-09-13T06:00:00-05:00,\nTencent Holdings (TCEHY) is the giant Chines...,Bear of the Day: Tencent (TCEHY),https://www.nasdaq.com/article/bear-of-the-day...
354,2018-09-13T06:00:00-05:00,\nTencent Holdings (TCEHY) is the giant Chines...,Bear of the Day: Tencent (TCEHY),https://www.nasdaq.com/article/bear-of-the-day...
369,2018-09-14T05:16:00-05:00,\n IMAX IMAX recently extended its partnersh...,IMAX Expands Presence in China with Multiple P...,https://www.nasdaq.com/article/imax-expands-pr...
372,2018-09-14T07:24:01-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...","3 Big Stock Charts for Friday: Comcast, Lockhe...",https://www.nasdaq.com/article/3-big-stock-cha...
370,2018-09-14T12:00:57-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",3 Tech Stocks With More Bite Than FANG,https://www.nasdaq.com/article/3-tech-stocks-w...


# Apply Datetime on the dataframe

In [81]:
gmt = pytz.timezone('GMT')
eastern = pytz.timezone('US/Eastern')

In [82]:
cleaned_frame["date_EST"] = cleaned_frame["datePublished"].apply(lambda x: dateutil.parser.parse(x).astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S'))


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [83]:
cleaned_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1322 entries, 0 to 1325
Data columns (total 5 columns):
datePublished    1322 non-null object
text             1322 non-null object
title            1322 non-null object
url              1322 non-null object
date_EST         1322 non-null object
dtypes: object(5)
memory usage: 62.0+ KB


In [84]:
cleaned_frame["date_UTC"] = cleaned_frame["datePublished"].apply(lambda x: datetime.utcfromtimestamp(dateutil.parser.parse(x).timestamp()).strftime('%Y-%m-%d %H:%M:%S'))


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [85]:
cleaned_frame

,datePublished,text,title,url,date_EST,date_UTC
0,2018-10-24T09:30:16-05:00,\nInvestors interested in stocks from the Broa...,TV vs. NFLX: Which Stock Is the Better Value O...,https://www.nasdaq.com/article/tv-vs-nflx-whic...,2018-10-24 10:30:16,2018-10-24 14:30:16
1,2018-10-22T05:45:18-05:00,"\nIn the latest trading session, Netflix (NFL...",Netflix (NFLX) Dips More Than Broader Markets:...,https://www.nasdaq.com/article/netflix-nflx-di...,2018-10-22 06:45:18,2018-10-22 10:45:18
2,2018-10-22T07:18:00-05:00,"\n For Immediate Release \nChicago, IL -Octob...",The Zacks Analyst Blog Highlights: Johnson & J...,https://www.nasdaq.com/article/the-zacks-analy...,2018-10-22 08:18:00,2018-10-22 12:18:00
3,2018-10-24T07:04:00-05:00,"\n For Immediate Release \nChicago, IL -Octob...","The Zacks Analyst Blog Highlights: Oracle, Net...",https://www.nasdaq.com/article/the-zacks-analy...,2018-10-24 08:04:00,2018-10-24 12:04:00
4,2018-10-23T02:08:11-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",SiriusXM Stock Could Bounce Back After Earnings,https://www.nasdaq.com/article/siriusxm-stock-...,2018-10-23 03:08:11,2018-10-23 07:08:11
5,2018-10-21T05:10:00-05:00,\nIf for no other reason than the fickle natu...,2 No-Brainer Stocks to Buy in the Entertainmen...,https://www.nasdaq.com/article/2-no-brainer-st...,2018-10-21 06:10:00,2018-10-21 10:10:00
6,2018-10-25T02:05:00-05:00,\nShares of Facebook FB opened higher Thursd...,Is It Finally Time to Buy Facebook (FB) Stock ...,https://www.nasdaq.com/article/is-it-finally-t...,2018-10-25 03:05:00,2018-10-25 07:05:00
7,2018-10-22T09:07:27-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...","3 Stock Charts for Monday: Netflix, PPL and Nv...",https://www.nasdaq.com/article/3-stock-charts-...,2018-10-22 10:07:27,2018-10-22 14:07:27
8,2018-10-19T03:06:21-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",7 Amazon-Proof Stocks to Consider Adding to Yo...,https://www.nasdaq.com/article/7-amazon-proof-...,2018-10-19 04:06:21,2018-10-19 08:06:21
9,2018-10-25T01:58:56-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",AT&T Stock Can Prove the Bears Wrong,https://www.nasdaq.com/article/att-stock-can-p...,2018-10-25 02:58:56,2018-10-25 06:58:56


In [229]:
cleaned_frame = cleaned_frame.sort_values(by="datePublished", ascending=True)

In [230]:
cleaned_frame

,datePublished,text,title,url,date_EST,date_UTC
665,2018-10-30T04:09:00-05:00,\nBy Munsif Vengattil and Paresh Dave\nOct 30 ...,"Facebook shares stung by slow user, sales growth",https://www.nasdaq.com/article/facebook-shares...,2018-10-30 05:09:00,2018-10-30 09:09:00
658,2018-10-30T05:31:00-05:00,"\nNEW YORK, Oct 30 (Reuters) - Deliv Inc, an U...",Same-day delivery company Deliv raises $40 mln...,https://www.nasdaq.com/article/sameday-deliver...,2018-10-30 06:31:00,2018-10-30 10:31:00
657,2018-10-30T07:04:00-05:00,\nBy Sonam Rai\nOct 30 (Reuters) - EBay Inc b...,"EBay beats qtrly profit on cost cut efforts, s...",https://www.nasdaq.com/article/ebay-beats-qtrl...,2018-10-30 08:04:00,2018-10-30 12:04:00
673,2018-10-31T01:04:00-05:00,\nBy Shreyashi Sanyal\nOct 31 (Reuters) - U....,"Tech resurgence, earnings lift mood at end of ...",https://www.nasdaq.com/article/tech-resurgence...,2018-10-31 02:04:00,2018-10-31 06:04:00
650,2018-10-31T01:04:00-05:00,"\n (For a live blog on the U.S. stock market, ...","US STOCKS-Tech resurgence, earnings lift mood ...",https://www.nasdaq.com/article/us-stockstech-r...,2018-10-31 02:04:00,2018-10-31 06:04:00
688,2018-10-31T01:28:00-05:00,\nThe Day Ahead newsletter:\nThe Morning News ...,"U.S. STOCKS ON THE MOVE-GM, FAANG stocks, Kell...",https://www.nasdaq.com/article/us-stocks-on-th...,2018-10-31 02:28:00,2018-10-31 06:28:00
653,2018-10-31T02:39:00-05:00,"\nBy Caroline Valetkevitch\nNEW YORK, Oct 31 (...",Wall St jumps as tech bounces back at end of b...,https://www.nasdaq.com/article/wall-st-jumps-a...,2018-10-31 03:39:00,2018-10-31 07:39:00
679,2018-10-31T02:39:00-05:00,"\n (For a live blog on the U.S. stock market, ...",US STOCKS-Wall St jumps as tech bounces back a...,https://www.nasdaq.com/article/us-stockswall-s...,2018-10-31 03:39:00,2018-10-31 07:39:00
663,2018-10-31T04:45:00-05:00,"\n (For a live blog on the U.S. stock market, ...",US STOCKS-Wall St ends last day of haunted Oct...,https://www.nasdaq.com/article/us-stockswall-s...,2018-10-31 05:45:00,2018-10-31 09:45:00
687,2018-10-31T06:38:00-05:00,\n (RTTNews.com) - Western Union Co. (WU) said...,"Western Union, Amazon To Unveil New Way To Pay",https://www.nasdaq.com/article/western-union-a...,2018-10-31 07:38:00,2018-10-31 11:38:00


In [231]:
cleaned_frame = cleaned_frame.reset_index()
cleaned_frame = cleaned_frame.drop(labels="index", axis=1)

In [232]:
cleaned_frame

,datePublished,text,title,url,date_EST,date_UTC
0,2018-10-30T04:09:00-05:00,\nBy Munsif Vengattil and Paresh Dave\nOct 30 ...,"Facebook shares stung by slow user, sales growth",https://www.nasdaq.com/article/facebook-shares...,2018-10-30 05:09:00,2018-10-30 09:09:00
1,2018-10-30T05:31:00-05:00,"\nNEW YORK, Oct 30 (Reuters) - Deliv Inc, an U...",Same-day delivery company Deliv raises $40 mln...,https://www.nasdaq.com/article/sameday-deliver...,2018-10-30 06:31:00,2018-10-30 10:31:00
2,2018-10-30T07:04:00-05:00,\nBy Sonam Rai\nOct 30 (Reuters) - EBay Inc b...,"EBay beats qtrly profit on cost cut efforts, s...",https://www.nasdaq.com/article/ebay-beats-qtrl...,2018-10-30 08:04:00,2018-10-30 12:04:00
3,2018-10-31T01:04:00-05:00,\nBy Shreyashi Sanyal\nOct 31 (Reuters) - U....,"Tech resurgence, earnings lift mood at end of ...",https://www.nasdaq.com/article/tech-resurgence...,2018-10-31 02:04:00,2018-10-31 06:04:00
4,2018-10-31T01:04:00-05:00,"\n (For a live blog on the U.S. stock market, ...","US STOCKS-Tech resurgence, earnings lift mood ...",https://www.nasdaq.com/article/us-stockstech-r...,2018-10-31 02:04:00,2018-10-31 06:04:00
5,2018-10-31T01:28:00-05:00,\nThe Day Ahead newsletter:\nThe Morning News ...,"U.S. STOCKS ON THE MOVE-GM, FAANG stocks, Kell...",https://www.nasdaq.com/article/us-stocks-on-th...,2018-10-31 02:28:00,2018-10-31 06:28:00
6,2018-10-31T02:39:00-05:00,"\nBy Caroline Valetkevitch\nNEW YORK, Oct 31 (...",Wall St jumps as tech bounces back at end of b...,https://www.nasdaq.com/article/wall-st-jumps-a...,2018-10-31 03:39:00,2018-10-31 07:39:00
7,2018-10-31T02:39:00-05:00,"\n (For a live blog on the U.S. stock market, ...",US STOCKS-Wall St jumps as tech bounces back a...,https://www.nasdaq.com/article/us-stockswall-s...,2018-10-31 03:39:00,2018-10-31 07:39:00
8,2018-10-31T04:45:00-05:00,"\n (For a live blog on the U.S. stock market, ...",US STOCKS-Wall St ends last day of haunted Oct...,https://www.nasdaq.com/article/us-stockswall-s...,2018-10-31 05:45:00,2018-10-31 09:45:00
9,2018-10-31T06:38:00-05:00,\n (RTTNews.com) - Western Union Co. (WU) said...,"Western Union, Amazon To Unveil New Way To Pay",https://www.nasdaq.com/article/western-union-a...,2018-10-31 07:38:00,2018-10-31 11:38:00


In [86]:
cleaned_frame.to_csv("FB_nasdaq_news_merged.csv", encoding='utf-8')

# Experment of Datetime

In [103]:
now_est = datetime.now(timezone('EST'))
print(now_est)

2019-01-27 22:33:14.105428-05:00


In [104]:
now_new_york = now_est.astimezone(timezone('America/New_York'))

In [106]:
now_new_york.timestamp()

1548646394.105428

In [129]:
_times = datetime.utcfromtimestamp(1548522180.0).strftime('%Y-%m-%d %H:%M:%S')

In [130]:
_times

'2019-01-26 17:03:00'

In [137]:
gmt = pytz.timezone('GMT')
eastern = pytz.timezone('US/Eastern')

#dategmt = gmt.localize(t)
#print(dategmt)
dateeastern = t.astimezone(eastern)
print(dateeastern.strftime('%Y-%m-%d %H:%M:%S'))

2019-01-26 12:03:00


In [128]:
dateeastern.timestamp()

1548522180.0

In [184]:
frame = pd.read_csv("GOOG_nasdaq_news.csv",header=0)

In [185]:
frame.head()

,Unnamed: 0,datePublished,text,timestamp_decode,title,unix_timestamp,url
0,0,2018-08-18T01:00:00-05:00,\n Amazon (NASDAQ: AMZN) bought Twitch in ...,2018-08-18 13:00:00,Can Amazon Help Twitch Become a $1 Billion Bus...,1.534597e+09,https://www.nasdaq.com/article/can-amazon-help...
1,1,2018-11-18T10:23:00-05:00,"\nBy Patpicha Tanakasempipat\nBANGKOK, Nov 16 ...",2018-11-18 22:23:00,Thai proposal for all-powerful cyber agency al...,1.542580e+09,https://www.nasdaq.com/article/thai-proposal-f...
2,2,2018-10-19T09:46:43-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",2018-10-19 21:46:43,What Will Apple Reveal At Its October 30 Event?,1.539986e+09,https://www.nasdaq.com/article/what-will-apple...
3,3,2018-12-21T08:15:00-05:00,\n Amazon (NASDAQ: AMZN) wants to control ...,2018-12-21 20:15:00,3 Amazon Products on Sale This Holiday Season,1.545423e+09,https://www.nasdaq.com/article/3-amazon-produc...
4,4,2018-11-13T10:44:39-05:00,\nThe Shanghai Composite is down about 2...,2018-11-13 22:44:39,Buying on the Dip in China,1.542149e+09,https://www.nasdaq.com/article/buying-on-the-d...


In [188]:
frame = frame.drop(labels=["Unnamed: 0", "timestamp_decode", "unix_timestamp"], axis=1)

In [189]:
frame

,datePublished,text,title,url
0,2018-08-18T01:00:00-05:00,\n Amazon (NASDAQ: AMZN) bought Twitch in ...,Can Amazon Help Twitch Become a $1 Billion Bus...,https://www.nasdaq.com/article/can-amazon-help...
1,2018-11-18T10:23:00-05:00,"\nBy Patpicha Tanakasempipat\nBANGKOK, Nov 16 ...",Thai proposal for all-powerful cyber agency al...,https://www.nasdaq.com/article/thai-proposal-f...
2,2018-10-19T09:46:43-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",What Will Apple Reveal At Its October 30 Event?,https://www.nasdaq.com/article/what-will-apple...
3,2018-12-21T08:15:00-05:00,\n Amazon (NASDAQ: AMZN) wants to control ...,3 Amazon Products on Sale This Holiday Season,https://www.nasdaq.com/article/3-amazon-produc...
4,2018-11-13T10:44:39-05:00,\nThe Shanghai Composite is down about 2...,Buying on the Dip in China,https://www.nasdaq.com/article/buying-on-the-d...
5,2018-08-31T03:14:14-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",4 Stocks That Belong in the Trillion Dollar Club,https://www.nasdaq.com/article/4-stocks-that-b...
6,2018-08-24T11:31:31-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",How to Trade Alibaba Stock After the Earnings ...,https://www.nasdaq.com/article/how-to-trade-al...
7,2018-08-28T06:13:45-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...","Amazon Stock Could Burst Through $2,000 for a ...",https://www.nasdaq.com/article/amazon-stock-co...
8,2018-10-27T08:03:00-05:00,\nIt seems like every time investors turn aro...,Here's Amazon's Next $10 Billion Business,https://www.nasdaq.com/article/heres-amazons-n...
9,2018-06-07T06:20:51-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...","Amazon.com, Inc. Looks to Take Over France Wit...",https://www.nasdaq.com/article/amazoncom-inc-l...


In [193]:
frame = frame.sort_values(by="datePublished", ascending=False)

In [194]:
frame["date_EST"] = frame["datePublished"].apply(lambda x: dateutil.parser.parse(x).astimezone(eastern).strftime('%Y-%m-%d %H:%M:%S'))

In [195]:
frame["date_UTC"] = frame["datePublished"].apply(lambda x: datetime.utcfromtimestamp(dateutil.parser.parse(x).timestamp()).strftime('%Y-%m-%d %H:%M:%S'))

In [196]:
frame

,datePublished,text,title,url,date_EST,date_UTC
490,2019-01-24T11:12:04-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",The 1 Thing Alphabet Should Absolutely Do to B...,https://www.nasdaq.com/article/the-1-thing-alp...,2019-01-24 11:12:04,2019-01-24 16:12:04
233,2019-01-24T10:06:16-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",5 Reasons This Is the Time to Get into Google ...,https://www.nasdaq.com/article/5-reasons-this-...,2019-01-24 10:06:16,2019-01-24 15:06:16
219,2019-01-24T06:08:00-05:00,\nDiscussion of driverless cars is becoming m...,3 Top Driverless Car Stocks to Watch in January,https://www.nasdaq.com/article/3-top-driverles...,2019-01-24 06:08:00,2019-01-24 11:08:00
174,2019-01-24T05:16:05-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",International Business Machines Finally Has A ...,https://www.nasdaq.com/article/international-b...,2019-01-24 05:16:05,2019-01-24 10:16:05
73,2019-01-24T03:30:22-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",Apple’s 200+ Staff Cut for Project Titan Isn’t...,https://www.nasdaq.com/article/apples-200-staf...,2019-01-24 03:30:22,2019-01-24 08:30:22
107,2019-01-23T12:02:52-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",Wall Street Is More Upbeat on Facebook Stock A...,https://www.nasdaq.com/article/wall-street-is-...,2019-01-23 12:02:52,2019-01-23 17:02:52
649,2019-01-23T12:01:49-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",Why GOOGL Stock Investors Should Welcome More ...,https://www.nasdaq.com/article/why-googl-stock...,2019-01-23 12:01:49,2019-01-23 17:01:49
1011,2019-01-23T10:09:29-05:00,"\nInvestorPlace - Stock Market News, Stock Adv...",Microsoft Announces New Products to Take Back ...,https://www.nasdaq.com/article/microsoft-annou...,2019-01-23 10:09:29,2019-01-23 15:09:29
647,2019-01-23T09:22:00-05:00,\n Alphabet 's (NASDAQ: GOOG) (NASDAQ: GOO...,Google Tries to Stay Relevant in Smartwatches ...,https://www.nasdaq.com/article/google-tries-to...,2019-01-23 09:22:00,2019-01-23 14:22:00
420,2019-01-22T10:51:30-05:00,Comparing units outstanding versus one week ag...,"XLC, DSTL: Big ETF Inflows",https://www.nasdaq.com/article/xlc-dstl-big-et...,2019-01-22 10:51:30,2019-01-22 15:51:30


In [199]:
frame["url"][490]

'https://www.nasdaq.com/article/the-1-thing-alphabet-should-absolutely-do-to-boost-googl-stock-cm1086920'

In [209]:
frame.to_csv("GOOG_nasdaq_news_cleaned.csv", encoding='utf-8')

In [203]:
frame = frame.sort_values(by="datePublished", ascending=True)

In [206]:
frame = frame.reset_index()

In [208]:
frame = frame.reset_index()
frame = frame.drop(labels="index", axis=1)